In [1]:
from datetime import date, timedelta
import os
import dill as pickle
import sys
import pandas as pd
import numpy as np
sys.path.append("..")

from abautomator.visualizer import BasicVisualizer, AbsDiffVisualizer, RelDiffVisualizer, utils
from abautomator import config, describer, analyzer

from bokeh.io import output_notebook
from bokeh.plotting import show, save
# from bokeh.models import MultiChoice, CustomJS, Button
from bokeh.layouts import column, row

output_notebook()
pd.set_option('display.max_colwidth', None)

Loading BokehJS ...

In [2]:
EXP_NAME = "ProtedtUpSellExp20224023_"
CTRL_NAME = 'ctrl'
COND_ORDER = ["tx_w_protect_upsell"]

In [3]:
def filter_coll_metrics(coll, column, value):
    for metric in coll.metrics:
        metric.user_metric_df = metric.user_metric_df[metric.user_metric_df[column] == value]
    
    return coll

In [4]:
local_coll = pickle.load(
    open(os.path.join("..", "tests", "cache", f"{EXP_NAME}_coll.p"), "rb" )
)

In [5]:
import copy

for mapping in [
        (None, "all"),
#         ("device_type","android"),
#         ("device_type", "ios"),
#         ("user_type","new"),
#         ("user_type", "existing"),
    ]:
    curr_coll = copy.deepcopy(local_coll)
    column, value = mapping
    if value != "all":
        column, value = mapping
        curr_coll = filter_coll_metrics(curr_coll, column=column, value=value)

    print(f"describing data for {value}")
    desc = describer.Describer(
        metrics=curr_coll.metrics
    )
    outcomes_dict = desc.describe_data(exp_name=EXP_NAME)
    
    print(f"analyzing data for {value}")
    analy =  analyzer.Analyzer(
        outcomes=outcomes_dict,
        ctrl_name=CTRL_NAME,
    )

    pickle.dump(
        analy, open(f"{EXP_NAME}_analy_{value}.p", "wb" )
    )

describing data for all
analyzing data for all


In [6]:
PRIMARY_METRIC_LIST = [
    'n_feed_views',
    'pct_feed_views',
    'n_feed_taps',
    'pct_feed_taps',
    'n_all_feed_shares',
    'pct_all_feed_shares',
    'n_direct_feed_shares',
    'pct_direct_feed_shares',
    'n_indirect_feed_shares',
    'pct_indirect_feed_shares',
    #  'n_signup_activation',
    'pct_signup_activation',
    'n_trial_starts',
    'n_protect_cancellations',
'                       '
]
 
SECONDARY_METRIC_LIST = [
 'n_all_sessions',
 'pct_all_sessions',
 'n_organic_sessions',
 'pct_organic_sessions',
 'n_push_driven_sessions',
 'pct_push_driven_sessions',
 'n_incident_views',
 'pct_incident_views',
 'n_incident_share_attempts',
 'pct_incident_share_attempts',
]

GUARDRAIL_METRIC_LIST = [
 'n_chats',
 'pct_chats',
 'n_friend_invites',
 'pct_friend_invites',
 'n_trial_starts',
 'pct_trial_starts',
]

In [7]:
def process_metric_set(metric_list, metric_list_label):
    for mapping in [
                (None, "all"),
#                 ("device_type","android"),
#                 ("device_type", "ios"),
#                 ("user_type","new"),
#                 ("user_type", "existing"),
            ]:
        _, value = mapping

        local_analy = pickle.load(
            open(f"{EXP_NAME}_analy_{value}.p", "rb" )
        )
        output_df = local_analy.get_rel_diff_confidence_intervals()
        print(output_df)
        df = utils.order_categories(output_df, metric_list, COND_ORDER)
        df = utils.remove_categories_w_no_order(df)
        source = utils.convert_df_to_source(df)

        vis = RelDiffVisualizer(source, title=f"{metric_list_label.title()} - {value.title()}")
        fig = vis.get_figure()
        to_show = vis.get_layout()
        show(to_show)

        save(to_show, f"/home/jovyan/work/notebooks/{value}-{metric_list_label.replace(' ', '-')}.html")

In [8]:
process_metric_set(PRIMARY_METRIC_LIST, "primary metrics")

NaN values found!


               exp_cond                       metric    tx_mean      tx_std  \
0   tx_w_protect_upsell                 n_feed_views  58.230490  185.393930   
1   tx_w_protect_upsell               pct_feed_views   0.995732    0.065193   
2   tx_w_protect_upsell                  n_feed_taps   1.360856    4.896637   
3   tx_w_protect_upsell                pct_feed_taps   0.423355    0.494091   
4   tx_w_protect_upsell            n_all_feed_shares   0.069615    0.789491   
5   tx_w_protect_upsell          pct_all_feed_shares   0.035267    0.184455   
6   tx_w_protect_upsell         n_direct_feed_shares   0.011657    0.338184   
7   tx_w_protect_upsell       pct_direct_feed_shares   0.009000    0.094443   
8   tx_w_protect_upsell       n_indirect_feed_shares   0.057958    0.644148   
9   tx_w_protect_upsell     pct_indirect_feed_shares   0.028147    0.165394   
10  tx_w_protect_upsell          n_signup_activation   0.016138    0.126007   
11  tx_w_protect_upsell        pct_signup_activation

/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


In [20]:
process_metric_set(SECONDARY_METRIC_LIST, "secondary metrics")

NaN values found!


/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
NaN values found!


/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
NaN values found!


/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
NaN values found!


/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
NaN values found!


/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


In [21]:
process_metric_set(GUARDRAIL_METRIC_LIST, "guardrail metrics")

NaN values found!


/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
NaN values found!


/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
NaN values found!


/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
NaN values found!


/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
NaN values found!


/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
